In [1]:
import pandas as pd
import json
from bs4 import BeautifulSoup
import numpy as np

In [2]:
with open('./source/questionnaire.json') as json_file:
    data = json.load(json_file)
    data = data['top_container']

In [3]:
country_key = 43380772

In [4]:
all_childs = []

def getAllChilds(x, pid=None, p=None):
    qid = None
    text = None
    dependencies = None
    if 'q_no' in x:
        qid = str(x['q_no'])
    if p:
        if 'q_no' in p:
            qid = str(p['q_no'])
        if 'q_no' in x:
            qid = str(x['q_no'])
    if 'text' in x:
        text = x['text']
        if text == " ":
            text = p['text']
    if 'title' in x:
        text = x['title']
    if pid:
        pid = str(pid)
    if 'dependencies' in x:
        dependencies = json.loads(x['dependencies'])
        dependencies = dependencies[0]
        rule = int(dependencies['rule'])
        parent = [o for o in p['options'] if o['value'] == rule]
        pid = str(parent[0]['id'])
    if text:
        text = BeautifulSoup(text)
        text = text.get_text()
        all_childs.append({
            "id": x['id'],
            "name": text.replace('\xa0',' ').replace('\n','').replace('  ',' '),
            "pid": pid,
            "qid": qid,
            "type": x["class"],
        })
    if 'options' in x:
        for z in x['options']:
            getAllChilds(z, x['id'], x)
    if 'children' in x:
        for z in x['children']:
            getAllChilds(z, x['id'], x)

In [5]:
for d in data['children']:
    getAllChilds(d)

In [6]:
df = pd.DataFrame(all_childs)

In [7]:
df = df[~df['type'].isin(['Page','Section','Note'])]

In [8]:
df['qpar'] = df['qid'].apply(lambda x: x.split('.')[0]).astype('int')

In [9]:
df.to_csv('test.csv')

In [10]:
resp = pd.read_csv('./source/data-production_2020-06-29.csv')

In [11]:
q_rename = {}
for qs in list(resp):
    qid = qs.split(' ')[0]
    if qid[-1] == ".":
        qid = qid[:-1]
        qname = '. '.join(qs.split('. ')[1:])
        ids = df[(df['qid']==qid)]
        ids = ids[~(ids['type']=='SelectionOption')]
        try:
            if qname in list(ids['name'])[0]:
                qid = list(ids['id'])[0]
        except:
            pass
    q_rename.update({qs:qid})

In [12]:
records = resp.rename(columns=q_rename)

In [13]:
non_var = ['StartDate','CompletionDate','Unique', country_key]
results = []
for a in records.to_dict('records'):
    for b in a:
        if b not in non_var:
            opt = df[(df['pid'] == str(b)) & (df['type'] == 'SelectionOption')]
            values = []
            for c in opt.to_dict('records'):
                if c['name'] in str(a[b]):
                    values.append(c['id'])
            if len(values) == 0:
                values = np.nan
            if values == values:
                results.append({'uuid':a['Unique'],'qid':b,'values':values,'countries':a[country_key]})

In [14]:
res = pd.DataFrame(results)

In [15]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, query
import numpy as np
from app.models import Countries, Groups, Values, CountryGroups, Datapoints, DatapointCountries, DatapointValues
from app.connection import engine_url, write_data

In [16]:
engine_url = engine_url()
engine = create_engine(engine_url)
session = sessionmaker(bind=engine)()

In [17]:
values = df.rename(columns={'id':'code','pid':'parent_code'})
values = values[['name','code','parent_code','type']]

In [18]:
question_list = list(res['qid'].unique())

In [19]:
for r in res['values'].to_list():
    for z in r:
        if z not in question_list:
            question_list.append(z)

In [20]:
values = values[values['code'].isin(question_list)]

In [21]:
def get_parent(x):
    parent_id = x['parent_code']
    if parent_id:
        parent = values[values['code'] == int(parent_id)]
        if parent.empty:
            return None
    return x['parent_code']

In [22]:
def get_grand_parent(x):
    parent_id = x['parent_code']
    if parent_id:
        parent = values[values['code'] == int(parent_id)]
        if not parent.empty:
            parent_id = parent.to_dict('records')[0]['parent_code']
            if parent_id:
                parent = values[values['code'] == int(parent_id)]
                if not parent.empty:
                    try:
                        return parent['code'][0]
                    except:
                        return parent_id
    return parent_id

In [23]:
values['parent_code'] = values.apply(get_parent, axis=1)
values['grand_parent'] = values.apply(get_grand_parent, axis=1)
values = values.reset_index()

In [24]:
excludes = values[(values['type'] == 'MultipleAnswer') & (values['parent_code'])]
excludes = excludes['index'].to_list()

In [25]:
values = values[~values['index'].isin(excludes)]
values['parent_code'] = values.apply(lambda x: x['grand_parent'] if x['grand_parent'] else x['parent_code'], axis=1)

In [27]:
for value in values.to_dict('records'):
    run = True
    if value['type'] == 'SelectionList' and value['parent_code'] != None:
        run = False
    if run:
        value.update({'parent_id':None,'description':None})
        parent = session.query(Values).filter(Values.code == value['parent_code']).first()
        if parent:
            value.update({'parent_id':parent.id})
        input_data = Values(value)
        write_data(session, input_data, value, False)

In [28]:
for uuid in set(list(records['Unique'])):
    datapoint = {'uuid':uuid}
    input_data = Datapoints(datapoint)
    write_data(session, input_data, datapoint, False)

In [29]:
for dc in records[['Unique', country_key]].to_dict('records'):
    datapoint = session.query(Datapoints).filter(Datapoints.uuid == dc['Unique']).first()
    if dc[country_key] == dc[country_key]:
        for c in dc[country_key].split(','):
            country = session.query(Countries).filter(Countries.name == c).first()
            if country:
                datapoint_country = {
                    'datapoint_id':datapoint.id,
                    'country_id': country.id
                }
                input_data = DatapointCountries(datapoint_country)
                write_data(session, input_data, datapoint_country, False)

In [30]:
for values in res.to_dict('records'):
    datapoint = session.query(Datapoints).filter(Datapoints.uuid == values['uuid']).first()
    for v in values['values']:
        value = session.query(Values).filter(Values.code == v).first()
        if value:
            datapoint_value = {
                'datapoint_id':datapoint.id,
                'value_id': value.id
            }
            input_data = DatapointValues(datapoint_value)
            write_data(session, input_data, datapoint_value, False)